In [4]:
# =============== 1) KURULUM ===============
!pip install -q medmnist
!pip install -q torchvision
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms

# Drive varsa
from google.colab import drive
drive.mount('/content/drive')
# ==========================================


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# =============== 2) VERİYİ AL ===============
from medmnist import INFO, ChestMNIST

# ChestMNIST bilgisi
info = INFO["chestmnist"]
task = info["task"]              # "multi-label, binary-class"
n_channels = info["n_channels"]  # 1
n_classes  = len(info["label"])  # 14

# veri setlerini indir
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),  # 1 kanal → 3 kanal
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_ds = ChestMNIST(split="train", download=True, transform=data_transform, as_rgb=False)
val_ds   = ChestMNIST(split="val",   download=True, transform=data_transform, as_rgb=False)
test_ds  = ChestMNIST(split="test",  download=True, transform=data_transform, as_rgb=False)

print(len(train_ds), len(val_ds), len(test_ds))  # kontrol
# ============================================


100%|██████████| 82.8M/82.8M [01:44<00:00, 791kB/s] 


78468 11219 22433


In [6]:
# =============== 2) VERİYİ AL ===============
from medmnist import INFO, ChestMNIST

# ChestMNIST bilgisi
info = INFO["chestmnist"]
task = info["task"]              # "multi-label, binary-class"
n_channels = info["n_channels"]  # 1
n_classes  = len(info["label"])  # 14

# veri setlerini indir
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),  # 1 kanal → 3 kanal
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_ds = ChestMNIST(split="train", download=True, transform=data_transform, as_rgb=False)
val_ds   = ChestMNIST(split="val",   download=True, transform=data_transform, as_rgb=False)
test_ds  = ChestMNIST(split="test",  download=True, transform=data_transform, as_rgb=False)

print(len(train_ds), len(val_ds), len(test_ds))  # kontrol
# ============================================


78468 11219 22433


In [7]:
# =============== 3) DATALOADER ===============
BATCH_SIZE = 64

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
# =============================================


In [8]:
# =============== 4) MODEL ===============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
in_feats = model.classifier.in_features
model.classifier = nn.Linear(in_feats, n_classes)   # 14 çıkış
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()  # çünkü chestmnist multi-label
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2
)

SAVE_DIR = "/content/drive/MyDrive/radiologyst"
os.makedirs(SAVE_DIR, exist_ok=True)
SAVE_PATH = os.path.join(SAVE_DIR, "chestmnist_best.pt")
# ===========================================


In [9]:
# =============== 5) EĞİTİM FONKSİYONLARI ===============
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total = 0.0
    for imgs, labels in loader:
        # medmnist label'ları shape (bs, 14, 1) gelebilir → sıkıştır
        labels = labels.squeeze().float()
        imgs   = imgs.to(device)
        labels = labels.to(device)

        out = model(imgs)
        loss = criterion(out, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += loss.item() * imgs.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total = 0.0
    for imgs, labels in loader:
        labels = labels.squeeze().float()
        imgs   = imgs.to(device)
        labels = labels.to(device)
        out = model(imgs)
        loss = criterion(out, labels)
        total += loss.item() * imgs.size(0)
    return total / len(loader.dataset)
# =========================================================


In [ ]:
# =============== 6) EĞİT ===============
MAX_EPOCHS = 15
PATIENCE   = 4
best_val   = float("inf")
no_imp     = 0

for epoch in range(1, MAX_EPOCHS+1):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss   = evaluate(model, val_loader,   criterion, device)
    scheduler.step(val_loss)

    print(f"[{epoch:02d}/{MAX_EPOCHS}] train={train_loss:.4f}  val={val_loss:.4f}")

    if val_loss < best_val - 1e-4:
        best_val = val_loss
        no_imp = 0
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"  -> iyileşme var, kaydedildi: {SAVE_PATH}")
    else:
        no_imp += 1
        print(f"  -> yok ({no_imp}/{PATIENCE})")

    if no_imp >= PATIENCE:
        print("early stopping")
        break

print("bitti. en iyi val:", best_val)
print("model:", SAVE_PATH)
# ============================================


[01/15] train=0.1730  val=0.1576
  -> iyileşme var, kaydedildi: /content/drive/MyDrive/radiologyst/chestmnist_best.pt


In [ ]:
# =============== 7) TEST / ÖRNEK TAHMİN ===============
@torch.no_grad()
def predict_one(model, ds, idx=0):
    model.eval()
    img, label = ds[idx]
    x = img.unsqueeze(0).to(device)
    out = model(x)
    probs = torch.sigmoid(out).cpu().squeeze(0).tolist()
    return probs, label

probs, label = predict_one(model, test_ds, idx=0)
print("ilk test görüntüsü gerçek label:", label.squeeze().tolist())
print("model olasılıkları (14 hastalık):", [round(p,4) for p in probs])
# ===========================================


In [ ]:
import os
from PIL import Image
import torch

# kaç tane örnek kaydedelim?
N = 10  # istersen arttır

out_dir = "/content/drive/MyDrive/chestmnist_samples"
os.makedirs(out_dir, exist_ok=True)

# ChestMNIST label'ları: 14 boyutlu multi-label vektör
# train_ds zaten daha önce oluşturduğumuz dataset
for i in range(N):
    img, label = train_ds[i]  # img: Tensor (3,224,224), label: (14,)
    # tensordan PIL'e çevir
    # önce unnormalize edelim:
    img_np = img.permute(1, 2, 0).cpu().numpy()
    # normalize geri alma (basit, tam değil ama görünmesi için yeter):
    img_np = (img_np * 255).clip(0, 255).astype("uint8")
    pil = Image.fromarray(img_np)

    # label'ı dosya adına da koyabiliriz
    # label tensor olabilir, listeye çevir
    label_list = label.squeeze().tolist()
    # hangi etiketler 1 ise onları yaz
    active_idxs = [str(j) for j, v in enumerate(label_list) if v == 1]

    fname = f"sample_{i:02d}_" + ("_".join(active_idxs) if active_idxs else "none") + ".png"
    pil.save(os.path.join(out_dir, fname))

print(f"{N} örnek kaydedildi -> {out_dir}")
